In [1]:
%cd /content/drive/My Drive/Signate/JLeague_audience_predict/JLeague

/content/drive/My Drive/Signate/JLeague_audience_predict/JLeague


In [0]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from pandas import Series
import re
import scipy
import scipy.stats

In [3]:
#訓練データ
input_dataframe = pd.read_csv('./data/input_dataframe.csv')
input_dataframe

,id,year,time_id,stage_id,gameday_id,capa,tv_num,home_id,away_id,temperature,humidity_id,weather_id,y
0,13994,2012.0,1404.0,1.0,310.0,19694.0,4.0,17.0,39.0,3.8,66.0,27.0,18250.0
1,13995,2012.0,1404.0,1.0,310.0,40000.0,4.0,23.0,37.0,12.4,43.0,1.0,24316.0
2,13996,2012.0,1404.0,1.0,310.0,21000.0,4.0,6.0,21.0,11.3,41.0,9.0,17066.0
3,13997,2012.0,1406.0,1.0,310.0,50000.0,4.0,10.0,36.0,11.4,52.0,21.0,29603.0
4,13998,2012.0,1404.0,1.0,310.0,39232.0,4.0,8.0,14.0,22.5,32.0,1.0,25353.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1948,16234,2014.0,1904.0,2.0,730.0,15165.0,2.0,14.0,4.0,26.7,80.0,2.0,7842.0
1949,16235,2014.0,1904.0,2.0,730.0,20000.0,2.0,40.0,16.0,28.0,78.0,2.0,5212.0
1950,16236,2014.0,1903.0,2.0,730.0,22563.0,2.0,1.0,38.0,25.4,83.0,13.0,3883.0
1951,16237,2014.0,1903.0,2.0,730.0,40000.0,2.0,24.0,32.0,25.4,84.0,33.0,5028.0


In [4]:
input_np = input_dataframe[['year', 'time_id', 'stage_id',	'gameday_id',	'capa',	'tv_num',	'home_id',	'away_id',	'temperature',	'humidity_id',	'weather_id']].values.astype(np.float32)
input_np

array([[2.012e+03, 1.404e+03, 1.000e+00, ..., 3.800e+00, 6.600e+01,
        2.700e+01],
       [2.012e+03, 1.404e+03, 1.000e+00, ..., 1.240e+01, 4.300e+01,
        1.000e+00],
       [2.012e+03, 1.404e+03, 1.000e+00, ..., 1.130e+01, 4.100e+01,
        9.000e+00],
       ...,
       [2.014e+03, 1.903e+03, 2.000e+00, ..., 2.540e+01, 8.300e+01,
        1.300e+01],
       [2.014e+03, 1.903e+03, 2.000e+00, ..., 2.540e+01, 8.400e+01,
        3.300e+01],
       [2.014e+03, 1.934e+03, 2.000e+00, ..., 2.790e+01, 6.100e+01,
        2.000e+00]], dtype=float32)

In [5]:
input_np.shape[0]

1953

In [6]:
input_sci = scipy.stats.zscore(input_np)
input_sci

array([[-1.0823061 , -1.0066838 , -1.2461706 , ..., -2.584851  ,
         0.2897443 ,  2.0210743 ],
       [-1.0823061 , -1.0066838 , -1.2461706 , ..., -1.2488431 ,
        -0.86325186, -0.90366316],
       [-1.0823061 , -1.0066838 , -1.2461706 , ..., -1.4197278 ,
        -0.9635124 , -0.00374392],
       ...,
       [ 1.5564593 ,  1.1479734 ,  0.8024583 , ...,  0.77070343,
         1.1419588 ,  0.4462157 ],
       [ 1.5564593 ,  1.1479734 ,  0.8024583 , ...,  0.77070343,
         1.1920891 ,  2.6960137 ],
       [ 1.5564593 ,  1.2818298 ,  0.8024583 , ...,  1.1590778 ,
         0.03909294, -0.7911733 ]], dtype=float32)

In [16]:
from sklearn import preprocessing
mm = preprocessing.MinMaxScaler()
input_sk = mm.fit_transform(input_np)
input_sk

array([[0.        , 0.22178984, 0.        , ..., 0.07317074, 0.62068963,
        0.76470584],
       [0.        , 0.22178984, 0.        , ..., 0.33536586, 0.3563218 ,
        0.        ],
       [0.        , 0.22178984, 0.        , ..., 0.30182928, 0.3333333 ,
        0.23529413],
       ...,
       [1.        , 0.86900127, 1.        , ..., 0.73170733, 0.81609195,
        0.3529412 ],
       [1.        , 0.86900127, 1.        , ..., 0.73170733, 0.8275862 ,
        0.9411765 ],
       [1.        , 0.9092089 , 1.        , ..., 0.80792683, 0.56321836,
        0.02941176]], dtype=float32)

In [7]:
output_np = input_dataframe[['y']].values.astype(np.float32)
output_np

array([[18250.],
       [24316.],
       [17066.],
       ...,
       [ 3883.],
       [ 5028.],
       [ 2221.]], dtype=float32)

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

In [78]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        
        # 線形層 y = Wx + b
        self.fc1 = nn.Linear(11, 20)   
        self.fc2 = nn.Linear(20, 20)
        self.fc3 = nn.Linear(20, 1)

    def forward(self, x):

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]   # バッチの次元以外のすべての次元
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (fc1): Linear(in_features=11, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=1, bias=True)
)


In [0]:
import torch.optim as optim

criterion = nn.MSELoss()

optimizer = optim.SGD(net.parameters(), lr=0.00001, momentum=0.9)

In [80]:
for epoch in range(20):  # 訓練データを複数回(2周分)学習する

    running_loss = 0.0
    for i in range(0, input_np.shape[0]):
        # ローダからデータを取得する; データは [inputs, labels] の形で取得される
        # イテレータを使用していないように見えますが for の内部で使用されています。
        inputs = torch.tensor(input_sk[i:i+1], requires_grad=True)
        labels = torch.tensor(output_np[i:i+1])
        # 勾配を0に初期化する(逆伝播に備える)
        optimizer.zero_grad()

        # 順伝播 + 逆伝播 + 最適化(訓練)
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()


    print('epoch: {},loss: {}'.format(epoch+1, running_loss))

print('Finished Training')

epoch: 1,loss: 289581941995.797
epoch: 2,loss: 200815301715.3728
epoch: 3,loss: 162327557975.3669
epoch: 4,loss: 144455948056.39258
epoch: 5,loss: 136098792309.05357
epoch: 6,loss: 132152007701.99985
epoch: 7,loss: 130262230063.29243
epoch: 8,loss: 129340585887.84
epoch: 9,loss: 128880190826.47829
epoch: 10,loss: 128643339013.05908
epoch: 11,loss: 128517205232.2962
epoch: 12,loss: 128447482114.71356
epoch: 13,loss: 128407426711.2486
epoch: 14,loss: 128383615389.17928
epoch: 15,loss: 128369030718.74965
epoch: 16,loss: 128359803190.73358
epoch: 17,loss: 128353901963.94516
epoch: 18,loss: 128350098195.01633
epoch: 19,loss: 128347514814.99344
epoch: 20,loss: 128345821762.94232
Finished Training


In [81]:
output_np[1:2]

array([[24316.]], dtype=float32)

In [82]:
out = net(torch.tensor(input_np[1:2]))
out

tensor([[10467.9951]], grad_fn=<AddmmBackward>)

In [62]:
net.fc1.weight

Parameter containing:
tensor([[ 1.8397e-01, -3.5549e-02, -1.7899e-01,  2.2170e-01, -2.2685e-01,
         -1.7577e-01, -2.0200e-01, -1.6962e-01,  1.5731e-01, -2.2575e-01,
          2.1277e-01],
        [-2.5436e-01, -7.1524e-03,  1.1877e-01,  2.5127e-01, -2.9966e-01,
         -6.3639e-01, -2.2285e-01, -5.1265e-01, -5.0458e-01, -1.9709e-01,
         -3.2622e-01],
        [-3.0034e-01, -2.0586e-01, -1.9031e-01,  1.9479e-01, -2.1636e-02,
         -1.9892e-01, -2.4109e-01, -1.0817e-01,  1.1955e-01, -2.2164e-01,
         -2.0540e-02],
        [ 2.9609e-01, -6.4892e+04,  9.1537e-02, -1.0522e+03, -3.5377e+04,
         -5.2923e+04, -6.3099e+04, -2.4783e+03, -1.8371e+04, -6.4571e+04,
         -3.7424e+04],
        [ 2.4707e-01, -2.9980e-01, -1.7045e-01,  1.3985e-01,  7.1591e-03,
         -2.8362e-01, -1.4791e-01,  1.1281e-01,  2.8720e-01,  6.7090e-02,
         -2.0159e-01],
        [-1.6079e-01, -2.9965e-01,  2.4051e-02, -6.7162e-02,  6.3166e-02,
          6.5583e-02, -2.8848e-01, -2.0670e-01, -